In [1]:
pip install psycopg2 neo4j pandas


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 28.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import psycopg2
import pandas as pd

# PostgreSQL Connection
pg_conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="Sehun0412",
    host="localhost",
    port="5432"
)
pg_cursor = pg_conn.cursor()

# Test Query
pg_cursor.execute("SELECT COUNT(*) FROM amazon_products;")
result = pg_cursor.fetchone()

print("✅ PostgreSQL is connected! Total products:", result[0])


✅ PostgreSQL is connected! Total products: 548552


In [12]:
from neo4j import GraphDatabase

# Neo4j Connection
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "Sehun0412"))

def test_neo4j_connection():
    query = "MATCH (p:Product) RETURN count(p) AS total_products"
    with neo4j_driver.session() as session:
        result = session.run(query)
        total_products = result.single()["total_products"]
        print("✅ Neo4j is connected! Total products:", total_products)

test_neo4j_connection()

✅ Neo4j is connected! Total products: 548552


<h3>First Attempt -- only work when searching with exact title</h3>

In [13]:
def get_recommendations(product_title):
    print(f"🔍 Searching for recommendations for: {product_title}")

    # Step 1: Query Neo4j for related products
    query = """
    MATCH (p:Product {title: $title})-[:CO_PURCHASED_WITH]->(p2)
    RETURN p2.asin AS recommended_asin
    ORDER BY p2.sales_rank
    LIMIT 5
    """
    with neo4j_driver.session() as session:
        result = session.run(query, title=product_title)
        asin_list = [record["recommended_asin"] for record in result]

    print("🔍 ASINs returned from Neo4j:", asin_list)  # Debugging step

    if not asin_list:
        return "⚠ No recommendations found."

    # Step 2: Query PostgreSQL for product details
    sql_query = """
    SELECT asin, title, product_group, sales_rank
    FROM amazon_products
    WHERE asin IN %s
    """
    pg_cursor.execute(sql_query, (tuple(asin_list),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")  # Debugging step

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank"])

# Example Usage
product_to_recommend = "Your Five- and Six-Year-Old: As They Grow"  # Use an existing product title
recommendations = get_recommendations(product_to_recommend)

from IPython.display import display
display(recommendations)


🔍 Searching for recommendations for: Your Five- and Six-Year-Old: As They Grow
🔍 ASINs returned from Neo4j: ['0761521364', '0060922761', '0312264208', '0440506735', '0440506743']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006
1,0312264208,Your Three- and Four-Year-Old: As They Grow,Book,16161
2,0440506735,Your Five Year Old: Sunny and Serene,Book,20389
3,0440506743,Your Six-Year-Old : Loving and Defiant,Book,23158
4,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333


<h3>Second Attempt </h3>

In [17]:
def get_recommendations(search_input, search_by_asin=False):
    """
    Get product recommendations based on:
    - Keywords in the title (default)
    - ASIN (if search_by_asin=True)
    
    Recommendations are ranked by **average rating** from PostgreSQL.
    """
    if search_by_asin:
        print(f"🔍 Searching recommendations for ASIN: {search_input}")
        query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        RETURN p2.asin AS recommended_asin
        ORDER BY p2.sales_rank
        LIMIT 5
        """
    else:
        print(f"🔍 Searching recommendations for title containing: '{search_input}'")
        query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE toLower(p.title) CONTAINS toLower($title)
        RETURN p2.asin AS recommended_asin
        ORDER BY p2.sales_rank
        LIMIT 5
        """

    with neo4j_driver.session() as session:
        print("🟡 Running Neo4j Query...")
        result = session.run(query, {"title": search_input} if not search_by_asin else {"asin": search_input})

        asin_list = [record["recommended_asin"] for record in result]
        print("🔍 ASINs returned from Neo4j:", asin_list)

    if not asin_list:
        return "⚠ No recommendations found in Neo4j."

    # Step 2: Query PostgreSQL for product details, ranked by avg rating
    sql_query = """
    SELECT p.asin, p.title, p.product_group, p.sales_rank, 
           COALESCE((SELECT AVG(r.rating) FROM amazon_reviews r WHERE r.asin = p.asin), 0) AS avg_rating
    FROM amazon_products p
    WHERE p.asin IN %s
    ORDER BY avg_rating DESC
    """
    pg_cursor.execute(sql_query, (tuple(asin_list),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])


In [18]:
# Example Usage: Search by Keyword
recommendations_by_title = get_recommendations("six-year")
from IPython.display import display
display(recommendations_by_title)

# Example Usage: Search by ASIN
recommendations_by_asin = get_recommendations("0761521364", search_by_asin=True)
display(recommendations_by_asin)

🔍 Searching recommendations for title containing: 'six-year'
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0380811960', '0761521364', '0761521364', '0060923288', '0060922761']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006,5.0000000000000000
1,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333,4.8666666666666667
2,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
3,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837


🔍 Searching recommendations for ASIN: 0761521364
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0380811960', '0060923288', '076112182X', '0071383018']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
1,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
2,076112182X,The Pocket Parent,Book,945,4.5000000000000000
3,0071383018,"Parenting the Strong-Willed Child, Revised and...",Book,2480,4.3333333333333333


<h3>Final Attempt -- Only Uses Neo4j for Recommendations/Fetches Ratings from PostgreSQL/Orders Results by Rating/Does Not Use Category-Based Recommendations </h3>

In [21]:
def get_recommendations(search_input, search_by_asin=False):
    """
    Get product recommendations based on:
    - Keywords in the title (default)
    - ASIN (if search_by_asin=True)
    
    Recommendations are ranked by **average rating** from PostgreSQL.
    """
    if search_by_asin:
        print(f"🔍 Searching recommendations for ASIN: {search_input}")
        query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        RETURN DISTINCT p2.asin AS recommended_asin, p2.sales_rank
        ORDER BY p2.sales_rank
        LIMIT 10
        """
    else:
        print(f"🔍 Searching recommendations for title containing: '{search_input}'")
        query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE toLower(p.title) CONTAINS toLower($title)
        RETURN DISTINCT p2.asin AS recommended_asin, p2.sales_rank
        ORDER BY p2.sales_rank
        LIMIT 10
        """

    with neo4j_driver.session() as session:
        print("🟡 Running Neo4j Query...")
        result = session.run(query, {"title": search_input} if not search_by_asin else {"asin": search_input})

        # Extract ASINs and remove duplicates
        asin_list = list(set(record["recommended_asin"] for record in result))
        print("🔍 ASINs returned from Neo4j:", asin_list)

    if not asin_list:
        return "⚠ No recommendations found in Neo4j."

    # Step 2: Query PostgreSQL for product details, ranked by avg rating
    sql_query = """
    SELECT p.asin, p.title, p.product_group, p.sales_rank, 
           COALESCE((SELECT AVG(r.rating) FROM amazon_reviews r WHERE r.asin = p.asin), 0) AS avg_rating
    FROM amazon_products p
    WHERE p.asin IN %s
    ORDER BY avg_rating DESC
    """
    pg_cursor.execute(sql_query, (tuple(asin_list),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])

In [22]:
# Example Usage: Search by Keyword
recommendations_by_title = get_recommendations("six-year")
from IPython.display import display
display(recommendations_by_title)

# Example Usage: Search by ASIN
recommendations_by_asin = get_recommendations("0761521364", search_by_asin=True)
display(recommendations_by_asin)

🔍 Searching recommendations for title containing: 'six-year'
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0060922761', '0471346985', '0440506816', '0761521364', '0316777153', '0440506751', '0440506506', '0380811960', '0060923288', '0440506492']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006,5.0000000000000000
1,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333,4.8666666666666667
2,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
3,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
4,0316777153,The Family Nutrition Book: Everything You Need...,Book,15531,4.5000000000000000
5,0440506816,Your Eight Year Old : Lively and Outgoing,Book,14686,4.2857142857142857
6,0440506506,Your Seven-Year-Old : Life in a Minor Key,Book,15437,4.2500000000000000
7,0471346985,Quick Meals for Healthy Kids and Busy Parents ...,Book,7073,3.6000000000000000
8,0440506751,Your Four-Year-Old : Wild and Wonderful,Book,14352,3.5714285714285714
9,0440506492,Your Three-Year-Old : Friend or Enemy,Book,9540,3.0000000000000000


🔍 Searching recommendations for ASIN: 0761521364
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['076112182X', '0380811960', '0060923288', '0071383018']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
1,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
2,076112182X,The Pocket Parent,Book,945,4.5000000000000000
3,0071383018,"Parenting the Strong-Willed Child, Revised and...",Book,2480,4.3333333333333333


<h3>Final Attempt V2 -- Combines Neo4j (Co-Purchase) + PostgreSQL (Category-Based)/ Fetches Ratings from PostgreSQL /Orders Results by Rating & Sales Rank</h3>

In [23]:
import pandas as pd

def get_combined_recommendations(search_input, search_by_asin=False):
    """
    Get product recommendations based on:
    - Co-purchase graph relationships (Neo4j)
    - Category-based recommendations (PostgreSQL)
    - Rank by **average rating**
    """

    ### Step 1: Neo4j Query (Co-Purchase Recommendations) ###
    if search_by_asin:
        print(f"🔍 Searching recommendations for ASIN: {search_input}")
        neo4j_query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        RETURN DISTINCT p2.asin AS recommended_asin
        LIMIT 10
        """
    else:
        print(f"🔍 Searching recommendations for title containing: '{search_input}'")
        neo4j_query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE toLower(p.title) CONTAINS toLower($title)
        RETURN DISTINCT p2.asin AS recommended_asin
        LIMIT 10
        """

    with neo4j_driver.session() as session:
        print("🟡 Running Neo4j Query...")
        result = session.run(neo4j_query, {"title": search_input} if not search_by_asin else {"asin": search_input})

        # Extract ASINs from Neo4j
        neo4j_asins = list(set(record["recommended_asin"] for record in result))
        print("🔍 ASINs returned from Neo4j:", neo4j_asins)

    ### Step 2: PostgreSQL Query (Category-Based Recommendations + Ratings) ###
    if search_by_asin:
        category_query = """
        SELECT DISTINCT p2.asin
        FROM amazon_products p1
        JOIN amazon_products p2 ON p1.categories = p2.categories
        WHERE p1.asin = %s AND p1.asin <> p2.asin
        LIMIT 10
        """
    else:
        category_query = """
        SELECT DISTINCT p2.asin
        FROM amazon_products p1
        JOIN amazon_products p2 ON p1.categories = p2.categories
        WHERE p1.title ILIKE %s AND p1.asin <> p2.asin
        LIMIT 10
        """

    pg_cursor.execute(category_query, (search_input if search_by_asin else f"%{search_input}%",))
    category_asins = [row[0] for row in pg_cursor.fetchall()]
    print("🔍 ASINs returned from PostgreSQL (Category-based):", category_asins)

    ### Step 3: Combine Neo4j + PostgreSQL ASINs ###
    combined_asins = list(set(neo4j_asins + category_asins))
    if not combined_asins:
        return "⚠ No recommendations found in Neo4j or PostgreSQL."

    ### Step 4: Get Full Product Details & Rank by Avg Rating ###
    sql_query = """
    SELECT p.asin, p.title, p.product_group, p.sales_rank, 
           COALESCE((SELECT AVG(r.rating) FROM amazon_reviews r WHERE r.asin = p.asin), 0) AS avg_rating
    FROM amazon_products p
    WHERE p.asin IN %s
    ORDER BY avg_rating DESC, p.sales_rank ASC
    """
    pg_cursor.execute(sql_query, (tuple(combined_asins),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])


# Example Usage (Title Search)
recommendations = get_combined_recommendations("six-year")
from IPython.display import display
display(recommendations)

# Example Usage (ASIN Search)
recommendations_asin = get_combined_recommendations("0761521364", search_by_asin=True)
display(recommendations_asin)

🔍 Searching recommendations for title containing: 'six-year'
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0060922761', '0440506743', '0440506816', '0761521364', '0312264208', '0440506751', '0440506506', '0380811960', '0440506735', '0440506492']
🔍 ASINs returned from PostgreSQL (Category-based): ['0440506506', '044050676X', '0440506816', '0553379496', '060980751X', '0609807617', '076112182X', '0761514708', '0761520112', '0761521755']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006,5.0000000000000000
1,0312264208,Your Three- and Four-Year-Old: As They Grow,Book,16161,5.0000000000000000
2,044050676X,Your Nine Year Old : Thoughtful and Mysterious,Book,36421,5.0000000000000000
3,0761521755,Positive Time-Out : And Over 50 Ways to Avoid ...,Book,90088,5.0000000000000000
4,0609807617,The Parent's Problem Solver : Smart Solutions ...,Book,93720,4.9444444444444444
5,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333,4.8666666666666667
6,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
7,0761514708,"Positive Discipline A-Z, Revised and Expanded ...",Book,5268,4.6000000000000000
8,076112182X,The Pocket Parent,Book,945,4.5000000000000000
9,060980751X,The Angry Child : Regaining Control When Your ...,Book,179827,4.3750000000000000


🔍 Searching recommendations for ASIN: 0761521364
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['076112182X', '0380811960', '0060923288', '0071383018']
🔍 ASINs returned from PostgreSQL (Category-based): ['0849935652']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
1,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
2,076112182X,The Pocket Parent,Book,945,4.5000000000000000
3,0071383018,"Parenting the Strong-Willed Child, Revised and...",Book,2480,4.3333333333333333
4,0849935652,Kid Think,Book,1357520,0
